In [1]:
from google.colab import drive
import sys
drive.mount('/content/drive')

FOLDERNAME = 'test_ml/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


##Подключение к устройству



In [2]:
import torch

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cpu


##Датасеты
Загрузка

In [4]:
import my_dataset

path_from = "drive/MyDrive/test_ml/dataset/train.zip"
path_to = "drive/MyDrive/test_ml/dataset/"

loaded_data = my_dataset.DataLoad(path_from, path_to)
loaded_data.data.head()

,filename,label
0,dog.4.jpg,dog
1,dog.40.jpg,dog
2,dog.400.jpg,dog
3,dog.4000.jpg,dog
4,dog.4001.jpg,dog


Разбиение датасета на 3 выборки: train = 80%, validation = 10%, test = 10%

In [5]:
x_train, x_valid, x_test = loaded_data.split_data()

print('Размер тренировочного набора данных', x_train.shape)
print('Размер валидационного набора данных', x_valid.shape)
print('Размер тестового набора данных', x_test.shape)

Размер тренировочного набора данных (15467, 2)
Размер валидационного набора данных (1933, 2)
Размер тестового набора данных (1934, 2)


Аугментация

In [6]:
from my_dataset import CustomDataset

train_dataset = CustomDataset(data=x_train, img_dir=path_to + "train/", transform=my_dataset.train_augmentation)
valid_dataset = CustomDataset(data=x_valid, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)
test_dataset = CustomDataset(data=x_test, img_dir=path_to + "train/", transform=my_dataset.test_augmentation)

Разметка на батчи

In [7]:
from torch.utils.data import DataLoader

batch_size = 64 # размер батча
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Просмотр состояния картинки из датасета

In [10]:
print(len(train_dataset))
for image, label in train_dataset:
    print(image.shape)
    print(label)
    break


15467
torch.Size([128, 128, 3])
dog


In [12]:
# 15467 / 64 = 242 (размер датасета / размер батча)
print(len(train_loader))
for image, label in train_loader:
    print(image.shape)
    print(label.shape)
    break

242


KeyError: ignored

##Тренировка модели
Модель представляет из себя сверточную нейросеть.

In [13]:
import model
from train import train_model

net = model.Model().to(device)

loss_history, y_pred_train = train_model(train_loader, net, epochs=25, device=device)


KeyError: ignored

Работа с validation набором

Работа с test набором

##Оценка качества и вывод графиков
Оценка качества будет производиться по метрике f1.

In [ ]:
import sklearn.metrics as metrics

y_true_train = x_train['label']
result = metrics.f1_score(y_true_train, y_pred_train)

print("Оценка качества F1:", result)


##Вывод графиков
Функция потерь

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(20),dpi=200)

plt.title('График потерь',fontsize=15)
plt.xlabel('Итерации',fontsize=12)
plt.ylabel('Потери',fontsize=12)
plt.plot(loss_history)

plt.show()

Confusion matrix для validation набора

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

conf_matrix_valid = confusion_matrix(y_valid, y_pred_valid)
conf_matrix_labels_valid = pd.DataFrame(conf_matrix_valid, columns = ['Positive', 'Negative'], index = ['True', 'False'])
conf_matrix_labels_valid

Сonfusion matrix для test набора

In [ ]:
conf_matrix_test = confusion_matrix(y_test, y_pred_test)
conf_matrix_labels_test = pd.DataFrame(conf_matrix_test, columns = ['Positive', 'Negative'], index = ['True', 'False'])
conf_matrix_labels_test